In [ ]:
import csv
from parsel import Selector
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
import time,csv,re,io,sys,datetime,os,requests
from bs4 import BeautifulSoup
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import glob
import os
import shutil
#Put down your email id and password
email=''
pswd=''
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
#Code to login to linkedin
option = Options()

option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")

# Pass the argument 1 to allow and 2 to block notifications
option.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2 })
driver=webdriver.Chrome(executable_path=r"C:\Users\user\Anaconda3\Library\usr\bin\chromedriver",options=option)
time.sleep(3)
driver.maximize_window()
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")
driver.implicitly_wait(6)
driver.find_element_by_xpath('//*[@id="username"]').send_keys(email)
sleep(0.5)
driver.find_element_by_xpath('//*[@id="password"]').send_keys(pswd)
sleep(0.5)
driver.find_element_by_xpath("//button[@class='btn__primary--large from__button--floating']").click()
sleep(2)

In [ ]:
#Search in google for linkedin profiles
driver.get('https://www.google.com/')
search_query = driver.find_element_by_name('q')
search_query.send_keys('site:linkedin.com/in AND "Marketing,B2B,Brand,Commissions,Sales,Promotions" AND "Australia"')
search_query.send_keys(Keys.RETURN)
sleep(0.5)

In [ ]:
#Create a dataframe to store linkedin profile info
DATA=pd.DataFrame(columns=['Name','Summary','Experience_1','Experience_2','Education_1','Education_2','Skills','URL'])

In [ ]:
#Scroll till the end of the page
def scroll_till_end():
    try:
        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.END)
    except Exception as e:
        print(str(e))

In [ ]:
#Capture linkedin URL's from current page in google search and add it to a list
def urlappend():
    urls = driver.find_elements_by_xpath('//*[@class = "r"]/a[@href]')
    urls = [url.get_attribute('href') for url in urls]
    return urls
    sleep(0.5)

In [ ]:
#To move to next page
def next_page():
    driver.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
    time.sleep(4)  

In [ ]:
#Slowly scroll down and then up at a speed of 8 sec to activate elements in a page and capture them
def __scroll_down_page(speed=8):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
#capture linkedin URL's of public profiles from first 25 pages displayed in google search 
pages_to_scrape=25
URL=[]
for i in range(pages_to_scrape):
    try:
        scroll_till_end()
        URL.extend(urlappend())
        scroll_till_end()
        next_page()
        time.sleep(4)
    except Exception as e:
        continue  

In [ ]:
#Iterate through the URL list to visit every profile and capture text from Summary,Skills,first 2 Experience and Education sections
#Store each profile details in a dictionary and then append the same to a dataframe
DICTIONARY=dict()
for url in url:
    driver.get(url)
    sleep(3)

    sel = Selector(text = driver.page_source)

    name = sel.xpath('//*[@class = "inline t-24 t-black t-normal break-words"]/text()').extract_first().split()
    name = ' '.join(name)
    DICTIONARY['Name']=name
    DICTIONARY['URL']= driver.current_url
    try:
        link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "see more")))
        ActionChains(driver).move_to_element(link).perform()
        link.click()
        DICTIONARY['Summary']=driver.find_element_by_xpath('//section[@class = "artdeco-container-card pv-profile-section pv-about-section ember-view"]/p').text
    except:
        continue
    experience = driver.find_elements_by_xpath('//section[@id = "experience-section"]/ul//li')[:2]
    if experience:
        count=1
        for i in experience:
            DICTIONARY['Experience_'+str(count)]=i.text
            count+=1
    education = driver.find_elements_by_xpath('//section[@id = "education-section"]/ul//li')[:2]
    if education:
        count=1
        for i in education:
            DICTIONARY['Education_'+str(count)]=i.text
            count+=1
    skll=driver.find_elements_by_xpath('//span[@class = "pv-skill-category-entity__name-text t-16 t-black t-bold"]')
    if skll:
        DICTIONARY['Skills']=[i.text for i in skll]
    else:
        __scroll_down_page(speed=9)
        sleep(3)
        skll=driver.find_elements_by_xpath('//span[@class = "pv-skill-category-entity__name-text t-16 t-black t-bold"]')
        DICTIONARY['Skills']=[i.text for i in skll]
        experience = driver.find_elements_by_xpath('//section[@id = "experience-section"]/ul//li')[:2]
        if experience:
            count=1
            for i in experience:
                DICTIONARY['Experience_'+str(count)]=i.text
                count+=1
        education = driver.find_elements_by_xpath('//section[@id = "education-section"]/ul//li')[:2]
        if education:
            count=1
            for i in education:
                DICTIONARY['Education_'+str(count)]=i.text
                count+=1
    DATA=DATA.append(DICTIONARY,ignore_index=True)

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
driver.quit()

In [ ]:
DATA.shape

(147, 8)

In [ ]:
DATA.tail(1)

,Name,Summary,Experience_1,Experience_2,Education_1,Education_2,Skills,URL
3,Girish Ramadurai,"Senior Supply chain professional with over 17 years of rich experience in various aspects of supply chain which include Demand & Supply planning, Supply Network optimization, SIOP, Inventory management, Production planning, Customer order management, Fulfillment Operations, Procurement, Logistics & Warehouse operations, Contract manufacturing operations. IT/ERP solutions for supply chain.\n\nOver 10 years of people management experience in recruitment, coaching & development of SCM professionals, aligning the team towards the functional and organisational objectives. Setting individual development objectives, assigning tasks, Inspiring & motivating team members to continuous improvements and exceed in performance parameters, review performance, provide guidance and support in excel in their career.","Company Name\nDanfoss\nTotal Duration\n10 yrs\nTitle\nHead of Global Logistics, SY\nDates Employed\nJul 2018 – Present\nEmployment Duration\n1 yr 10 mos\nLocation\nChennai Area, India\nHandling Global Logistics functions for System Solutions (SY) Division in Danfoss. Driving Global Demand Planning, Sales Inventory & Operations Planning, MRP, Inbound Logistics, Contract Manufacturing Operations, Fulfillment & Outbound Logistics.\nTitle\nGlobal S&OP Manager\nDates Employed\nOct 2016 – Jun 2018\nEmployment Duration\n1 yr 9 mos\nLocation\nChennai Area, India\nResponsible for Global Sales & Operational Planning for SY Division at Danfoss.\nTitle\nSenior Project Manager-Supply Chain\nDates Employed\nJan 2016 – Sep 2016\nEmployment Duration\n9 mos\nLocation\nChennai\nResponsible for Global Supply Chain Projects related to Supply Network Optimization and Supply Chain Projects for system and Tools.\n\nAdditionally responsible for Global supply chain planning & operations & Sub-Contract manufacturing operations for one critical Product Line of Danfoss.\n…\nsee more\nTitle\nSenior Manager - Regional Fulfillment, Asiapacific\nDates Employed\nApr 2012 – Dec 2015\nEmployment Duration\n3 yrs 9 mos\n 1. Responsible for FG supply chain in APAC region – Danfoss Cooling Division.\nManaging the Finish Goods Availability for Customer order fulfilment.\nManaging FG sourcing & replenishment strategies, optimize inventory levels and safety stocks to achieve best in class service level, shortest lead time and least costs to serve in APAC.\n\n 2. Responsible for Global Demand Fulfilment of Danfoss Condensing Units Business\nContract Manufacturing operations of Condensing Units in India and supply to Global Markets\nDemand planning, Components supply management to Contract manufacturer.\nCondensing unit production planning & supply to Global markets from Contract manufacturer\n\n 3. Responsible for Danfoss India factory S&OP and demand planning, weekly rolling demand inputs to factory for production planning and allocation management of finished products.\n\n Leading a team of SCM professionals in APAC region which include Logistics Specialist, Fulfillment Analyst & SCM Consultants. Setting objectives, drive performances, guide and troubleshoot, review and feedback on performance.\n\n Single interface between Danfoss Cooling factories across globe, sales operations team in APAC.\n\n Responsible for optimize FG levels of DOI at Central Distribution center and local country warehouses at all APAC country and manage the reorder points & safety stocks levels.\n\n Coordination with sales organization in reduction of SLOB inventory and periodic review of inventory levels to avoid further addition to slow movers.\n\n Responsible for managing logistics cost & reducing cost to serve without impacting the availability & customer service level.\n\n Handling strategic projects on supply network optimization\n…\nsee more\nTitle\nManager- Regional Fulfillment-Asiapacific\nDates Employed\nMay 2010 – Apr 2012\nEmployment Duration\n2 yrs\nJoined Danfoss during organizatio

In [ ]:
#Export data from dataframe in to an excel file
DATA.to_excel(r"D:\LINKEDIN_DATA\APRIL_13_2020_EXCELDATA\seethu_profiles_1.xlsx")

In [ ]:
pd.set_option('display.max_colwidth', -1)